# Simulations to identify research gaps
For each disease, we simulate what would have been the mapping of RCTs within regions if the misclassification of RCTs towards groups of diseases was corrected, given the sensitivities and specificities of the classifier to identify each group of disease.

To estimate the performances of the classifier for each group of diseases, we dispose a test set with 2,763 trials manually classified towards the 27-class grouping of diseases used in this work. The test set is described at Atal et al. BMC Bioinformatics 2016.

The method used is based on the method presented at Fox et al. Int J Epidemiol 2005.

To do so, for each disease for which we found a local research gap we will:

1. Calculating sensitivity and specificity of the classifier to identify the disease and other studies relevant to the burden of diseases, and the number of success and number of trials to derive beta distributions
2. Doing N=60k times the following simulation
    * Randomly choose a sens and spec based on beta distribution for identifying the disease and identifying another disease (no correlation between sens and spec, neither between disease and another disease both)
    * Derive Positive and Negative Predictive Values (PPV and NPV) for each.
    * Simulate the correction of the classification based on PPVs and NPVs
    * Derive the proportion of RCTs concerning the disease among all RCTs concerning the burden of disease in the region
3. Derive 95% upper-bond simulation interval of the proportion of RCTs concerning the disease among all RCTs concerning the burden of diseases

## 2. Simulating correction of misclassification

In [1]:
data <- read.table('/media/igna/Elements/HotelDieu/Cochrane/Mapping_Cancer/Flowchart/database_all_diseases_final_ok.txt')
data <- data[!is.na(data$Sample),]
data <- data[data$Sample>=20 & data$Sample<=150000,]
N <- nrow(data)

In [2]:
#Nb of patients per region
nb_ctrs <- lapply(strsplit(as.character(data$Nb_ctr_per_reg),'&'),as.numeric)
RGs <-strsplit(as.character(data$Regions),'&')
pats <- data.frame(TrialID = rep(data$TrialID,sapply(nb_ctrs,length)),
                   Nb_ctrs = unlist(nb_ctrs),
                   Region = unlist(RGs),
                   Tot_sample = rep(data$Sample,sapply(nb_ctrs,length)))

In [3]:
pats$tot_ctrs <- rep(sapply(nb_ctrs,sum),sapply(nb_ctrs,length))
pats$sample_per_reg <- pats$Tot_sample*pats$Nb_ctrs/pats$tot_ctrs

In [5]:
Lgbd <- lapply(as.character(data$GBD28),function(x){as.numeric(unlist(strsplit(x,"&")))})
Lgbd <- lapply(Lgbd,function(x){x[x!=28]})

In [6]:
g <- sample(1:27,1)
   
    #which trials concern the disease
    is_dis <- sapply(Lgbd,function(x){g%in%x})


In [7]:
ddd <- pats[pats$TrialID%in%data$TrialID[is_dis],]

In [8]:
tapply(ddd$sample_per_reg,ddd$Region,sum)

Central Europe, Eastern Europe, and Central Asia 
                                       377866.42 
                                     High-income 
                                      3293044.87 
                     Latin America and Caribbean 
                                       119740.42 
                    North Africa and Middle East 
                                       164837.05 
                                      South Asia 
                                       113709.33 
           Southeast Asia, East Asia and Oceania 
                                       708460.79 
                             Sub-Saharian Africa 
                                        79465.12

In [10]:
regs <- sort(unique(unlist(strsplit(as.character(data$Regions),"&"))))

In [12]:
library(gdata)

gdata: read.xls support for 'XLS' (Excel 97-2004) files ENABLED.

gdata: read.xls support for 'XLSX' (Excel 2007+) files ENABLED.

Attaching package: ‘gdata’

The following object is masked from ‘package:stats’:

    nobs

The following object is masked from ‘package:utils’:

    object.size

The following object is masked from ‘package:base’:

    startsWith



In [13]:
 trs <- is_dis
                    dsp <- data.frame(nb_ctrs = unlist(nb_ctrs[trs]),
                    sample = rep(data$Sample[trs],sapply(nb_ctrs[trs],length)),
                    tt_nb_ctrs = rep(sapply(nb_ctrs[trs],sum),sapply(nb_ctrs[trs],length)))
                    dsp$reg <- unlist(RGs[trs])
                    dsp$reg <- factor(dsp$reg)
                    levels(dsp$reg) <- c(levels(dsp$reg),setdiff(levels(dsp$reg),regs))
                    dsp$reg <- reorder(dsp$reg,new.order=regs)
                    dsp$spl_ctrs <- (dsp$sample*dsp$nb_ctrs)/dsp$tt_nb_ctrs

In [15]:
tapply(dsp$spl_ctrs,dsp$reg,sum)

Central Europe, Eastern Europe, and Central Asia 
                                       377866.42 
                                     High-income 
                                      3293044.87 
                     Latin America and Caribbean 
                                       119740.42 
                    North Africa and Middle East 
                                       164837.05 
                                      South Asia 
                                       113709.33 
           Southeast Asia, East Asia and Oceania 
                                       708460.79 
                             Sub-Saharian Africa 
                                        79465.12

In [3]:
Lgbd <- lapply(as.character(data$GBD28),function(x){as.numeric(unlist(strsplit(x,"&")))})
Lgbd <- lapply(Lgbd,function(x){x[x!=28]})

In [20]:
PERF <- read.csv('Tables/Performances_per_27disease_data.csv')

In [21]:
#For all diseases, we will simulate the mapping across regions of trials concerning
#the disease or concerning other diseases
dis <- 1:27

In [6]:
library(gdata)

gdata: read.xls support for 'XLS' (Excel 97-2004) files ENABLED.

gdata: read.xls support for 'XLSX' (Excel 2007+) files ENABLED.

Attaching package: ‘gdata’

The following object is masked from ‘package:stats’:

    nobs

The following object is masked from ‘package:utils’:

    object.size

The following object is masked from ‘package:base’:

    startsWith



In [7]:
regs <- sort(unique(unlist(strsplit(as.character(data$Regions),"&"))))

In [16]:
#NK <- 60000
NK <- 5000
set.seed(7212)

In [ ]:
#For each disease
t0 <- proc.time()

#for(g in dis){

g <- 24
print(g)
    PERF_g <- PERF[PERF$dis==g,]
    
    #which trials concern the disease
    is_dis <- sapply(Lgbd,function(x){g%in%x})
    #which trials concern another disease
    is_oth <- sapply(Lgbd,function(x){sum(setdiff(1:27,g)%in%x)>0})

    #PPV et NPVs for finding the disease
    sens_r <- PERF_g$TP_Dis
    sens_n <- PERF_g$TP_Dis + PERF_g$FN_Dis
    spec_r <- PERF_g$TN_Dis
    spec_n <- PERF_g$TN_Dis + PERF_g$FP_Dis
    sens <- rbeta(NK,sens_r+1,sens_n-sens_r+1)
    spec <- rbeta(NK,spec_r+1,spec_n-spec_r+1)

    a_dis <- sum(is_dis)
    b_dis <- N-a_dis
    As <- (a_dis-(1-spec)*N)/(sens - (1-spec))
    Bs <- N-As
    T1 <- sens*As
    T0 <- spec*Bs
    F1 <- (1-spec)*Bs
    F0 <- (1-sens)*As
    PPV_dis <- T1/(T1+F1)
    NPV_dis <- T0/(T0+F0)

    #PPV and NPVs for finding another disease
    sens_r <- PERF_g$TP_Oth
    sens_n <- PERF_g$TP_Oth + PERF_g$FN_Oth
    spec_r <- PERF_g$TN_Oth
    spec_n <- PERF_g$TN_Oth + PERF_g$FP_Oth
    sens <- rbeta(NK,sens_r+1,sens_n-sens_r+1)
    spec <- rbeta(NK,spec_r+1,spec_n-spec_r+1)

    a_oth <- sum(is_oth)
    b_oth <- N-a_oth
    As <- (a_oth-(1-spec)*N)/(sens - (1-spec))
    Bs <- N-As
    T1 <- sens*As
    T0 <- spec*Bs
    F1 <- (1-spec)*Bs
    F0 <- (1-sens)*As
    PPV_oth <- T1/(T1+F1)
    NPV_oth <- T0/(T0+F0)

    #Some values of sens and spec may lead to impossible values of PPV or NPV (>1 or <0)
    #Depending on the case we will either suppress the iteration, either fix PPVs and NPVs to 0 or 1
    #The decision is made to be the most conservative as possible regarding our objective of 
    #upper-bounding the number and local proportion of RCTs concerning a disease in a region

    #Case 1: PPV_dis < 0 (and NPV_dis > 1), sens_dis and spec_dis are such that expected value
        #of nb_trials concerning disease is negative. Suppressing that iteration is conservative
        #in regard of our objective, while fixing PPV_dis = 0 is not conservative, as we fix the number
        #of RCTs concerning the disease equal to 0 in all regions
    case1 <- PPV_dis<0 | NPV_dis>1 
    if(sum(case1>0)) print(paste(c(g,"has",sum(case1),"case 1 false iterations which were suppressed"),collapse=" "))

    #Case 2: PPV_dis > 1 (and NPV_dis < 0), sens_dis and spec_dis are such that expected value
        #of nb_trials concerning disease is higher than the total number of trials. Suppressing that iteration is 
        #not conservative in regard of our objective, as an iteration giving a maximal number and proportion
        #of RCTs will be suppressed, while fixing PPV_dis = 1 and NPV_dis = 0 (ie nb trials dis = N) 
        #is conservative
    case2 <- PPV_dis>1 | NPV_dis<0 
    if(sum(case2>0)) print(paste(c(g,"has",sum(case2),"case 2 false iterations for which we fixed PPV_dis = 1"),collapse=" "))
        
    #Case 3: If PPV_oth < 0 (and NPV_oth > 1), sens_oth and spec_oth are such that the expected value
        #of nb_trials concerning other diseases is negative. Suppressing that iteration is not conservative
        #in regard of our objective, as that iteration would give that all trials concerning the burden of
        #diseases concern the disease of interest. We fixed PPV_oth = 0 and NPV_oth = 1
    case3 <- PPV_oth<0 | NPV_oth>1 
    if(sum(case3>0)) print(paste(c(g,"has",sum(case3),"case 3 false iterations for which we fixed PPV_oth = 0"),collapse=" "))

    #Case 4: PPV_oth > 1 (and NPV_oth < 0), sens_oth and spec_oth are such that expected value
        #of nb_trials concerning other diseases is higher than the total number of trials. 
        #Suppressing that iteration is conservative in regard of our objective, while fixing PPV_oth = 1
        #is not conservative, as we fix the number of RCTs concerning other diseases to be maximal, so the proportion
        #of RCTs concerning the disease among all RCTs concerning the burden is minimized
    case4 <- PPV_oth>1 | NPV_oth<0 
    if(sum(case4>0)) print(paste(c(g,"has",sum(case4),"case 4 false iterations which were suppressed"),collapse=" "))

    #If the number of suppressed iteration is higher than 10% of the total number of iterations, we skip
    #the disease
    if(sum(case1 | case4)>0.1*NK){ print(paste(c(g,
                                                     "has", 
                                                     sum(case1 | case4), 
                                                     "(too many) suppressed false iterations"
                                                    ),collapse=" "))
#                          next
                          }
    
 #   else print(paste(c(g,"has",sum(case1 | case4),"suppressed false iterations"
  #                                                  ),collapse=" "))  

    PPV_dis[case2] <- 1
    NPV_dis[case2] <- 0
    PPV_oth[case3] <- 0
    NPV_oth[case3] <- 1
    PPV_dis <- PPV_dis[!(case1 | case4)]
    NPV_dis <- NPV_dis[!(case1 | case4)]
    PPV_oth <- PPV_oth[!(case1 | case4)]
    NPV_oth <- NPV_oth[!(case1 | case4)]

    L <- list()
    #Simulation: reclassifying each trial
        for(k in 1:length(PPV_dis)){

            AR <- matrix(0, nrow=length(regs), ncol=2)
            tp_dis <- runif(a_dis)
            tn_dis <- runif(b_dis)
            recl_dis <- is_dis
            recl_dis[recl_dis==TRUE][tp_dis>PPV_dis[k]] <- FALSE
            recl_dis[recl_dis==FALSE][tn_dis>NPV_dis[k]] <- TRUE
            #Rq: we count all trials (even those with more than 3 diseases)
            #it is a conservative choice
            rt <- as.numeric(recl_dis)

            if(sum(recl_dis)==0) AR[,1] <- c(rep(0,7))
            else{   
                    dsp <- pats[pats$TrialID%in%data$TrialID[recl_dis],]
                    AR[,1] <-tapply(dsp$sample_per_reg,dsp$Region,sum)
            }
                
            #Oth_dis
            tp_oth <- runif(a_oth)
            tn_oth <- runif(b_oth)
            recl_oth <- is_oth
            recl_oth[recl_oth==TRUE][tp_oth>PPV_oth[k]] <- FALSE
            recl_oth[recl_oth==FALSE][tn_oth>NPV_oth[k]] <- TRUE
            rt <- rt + as.numeric(recl_oth)

            if(sum(rt)==0) AR[,2] <- c(rep(0,7))
            else{  
                    dsp <- pats[pats$TrialID%in%data$TrialID[rt],]
                    AR[,2] <-tapply(dsp$sample_per_reg,dsp$Region,sum)

            }

            L[[k]] <- AR

        }

    
    T <- do.call('rbind',L)
    write.table(T,paste(c("/media/igna/Elements/HotelDieu/Cochrane/Mapping_Cancer/Incertitude_mapping/Simulations/Patients_total_simulation_",as.character(PERF_g$dis),".txt"),collapse=""))

#}

t1 <- proc.time()
    
print(t1-t0)/60

[1] 24


In [ ]:
160*2/60

For 10,000 simulations it will take 5.3h approx

## 3. Deriving 95% upper bound simulation intervals
For the number of RCTs concerning each disease in each region, and the local proportion of RCTs for each disease in each region

In [ ]:
Mgbd <- read.table("/home/igna/Desktop/Programs GBD/Classifier_Trial_GBD/Databases/Taxonomy_DL/GBD_data/GBD_ICD.txt")

In [ ]:
#We supress injuries
Mgbd <- Mgbd[-28,]

In [ ]:
GBD <- read.table("/media/igna/Elements/HotelDieu/Cochrane/Mapping_Cancer/Tables/GBD_data_per_region_and_27_diseases_2005.txt")

In [18]:
dis <- 1:27

In [ ]:
regs <- names(GBD)[1:7]

In [ ]:
SM <- data.frame(Region = rep(regs,each=nrow(Mgbd)),
                Disease = rep(Mgbd$cause_name,times=length(regs)))

In [ ]:
SM$SimMn_NbRCTs <- NA
SM$Sim95_NbRCTs <- NA
SM$SimMn_PrRCTs <- NA
SM$Sim95_PrRCTs <- NA

In [ ]:
for(g in dis){

    T <- tryCatch(read.table(paste(c("/media/igna/Elements/HotelDieu/Cochrane/Mapping_Cancer/Incertitude_mapping/Simulations/Total_simulation_",
                            as.character(g),".txt"),collapse="")),error=NULL)

    if(length(T)!=0){

        #Mean and 95% upper-bound number of RCTs by simulation
        M <- matrix(T[,1],ncol=7,byrow=TRUE)
        SM$Sim95_NbRCTs[SM$Disease==Mgbd$cause_name[g]] <- apply(M,2,function(x){quantile(x,0.95)})
        SM$SimMn_NbRCTs[SM$Disease==Mgbd$cause_name[g]] <- apply(M,2,mean)

        #Mean and 95% upper-bound proportion of RCTs by simulation
        M <- matrix(T[,1]/T[,2],ncol=7,byrow=TRUE)
        SM$Sim95_PrRCTs[SM$Disease==Mgbd$cause_name[g]] <- apply(M,2,function(x){quantile(x,0.95)})
        SM$SimMn_PrRCTs[SM$Disease==Mgbd$cause_name[g]] <- apply(M,2,mean)
        
    }   
}

In [ ]:
write.table(SM,'Data/Simulations_Alldis_NbProp_Mn95_RCTs.txt')